### Processing and analyzing images
This notebook highlights some basic essentials with processing and analyzing simulated images with a mixed-field sample of simulated ERs and NRs

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm #progressbar
import matplotlib.pyplot as plt
'''Setting up some rcparameters for better size axis labels'''
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('axes', labelsize=16)
plt.rc('axes', titlesize=16)

### Descriptions of each column (for people who use ROOT you'll hear columns being called "branches") in our dataframe:

-**nHits**: Number of primary electron tracks in the simulation

-**x**, **y**, **z**: Primary track x, y, and z coordinates

-**t**: Relative time of ionization deposit. (x,y,z) are sorted in time-order

-**flag**: Not so important. Flag indicates the physical process with which the ionization electron was generated: 1=fluorescence; 2=pair production; 3=bremsstrahlung; 0=otherwise

-**truth_dir**: (x,y,z) unit vector of track

-**truth_energy**: Energy of the primary track

-**ionization_energy**: ionization energy in CF4 of the primary track. This is computed as 'nHits' x W where W = 34.2 eV

-**truth_theta**: truth zenith angle (w.r.t z-axis) determined by 'truth_dir'

-**truth_phi**: truth aximuthal angle (in xy plane w.r.t +x) determined by 'truth_dir'

-**drift_length**: Amount of drift simulated. Currently using a random-uniform distribution between 1cm and 2.5cm. TODO for Jeff: Make this adjustable in configuration.yaml

-**xdiff**, **ydiff**, **zdiff**: x,y, and z coordinates of ionization after applying diffusion over 'drift_length'

-**xamp**, **yamp**, **zamp**: The electrons from (xdiff,ydiff,zdiff) that align with the openings of a GEM hole

-**xcam**, **ycam**, **qcam**: (x,y) after applying gain and diffusion through the transfer gap, binned to the 2048 x 1152 camera dimensions. 'qcam' is the number of amplified electrons falling into the bin

-**xITO**, **zITO**, **qITO**: (x,z) after applying gain and diffusion through the transfer gap, *and induction gap* binned to 120 strips along x. z is quantified assuming 0.26um per bin. 'qITO' is the number of amplified electrons falling into the ITO bin

In [ ]:
'''Load data and combine into a single dataframe
you"ll have to change the directories to wherever you downloaded
the files to'''

ERs = pd.read_feather("data/20000ERs_2-11.8keV_processed.feather")
NRs = pd.read_feather("RCTRIM/output/F_maxE466.957_5489primaryTracks.feather")

'''NRs doesn"t have an nHits column, let"s add one'''
NRs['nHits'] = NRs['x'].apply(lambda x: len(x)) #Number of primary ionization electrons

In [ ]:
'''Before we combine we should check the columns in the ER dataframe that aren"t in the NR dataframe'''
print(f"Columns not in NR: {[col for col in ERs.columns if col not in NRs.columns]}")
print(f"Columns not in ER: {[col for col in NRs.columns if col not in ERs.columns]}")

In [ ]:
'''Lets make NR['species'] = 1 and ER['species'] = 0
You can use this as the truth label for NRs and ERs respectively when performing
ML studies'''

NRs['species'] = 1
ERs['species'] = 0

'''Now let"s drop the ER columns that aren"t in the NR dataframe'''
ERs = ERs.drop(columns = ['t','flag'])

In [ ]:
'''Now that the columns match, we can merge the dataframes'''
df = pd.concat([NRs,ERs]) #merge NRs and ERs
df = df.sample(frac = 1) #randomize entries of dataframe
df.index = [i for i in range(0,len(df))]

In [ ]:
'''Let"s plot the species breakdown'''
fig,ax = plt.subplots()
ax.hist(df['species'],bins=2,range=(0,1))
ax.set_xticks([0.25,0.75])
ax.set_xticklabels(['ER','NR'])

In [ ]:
"""Now let's look at energy distributions"""
plt.hist(df.query('species == 0')['ionizationE'],bins=899,range = (2,450),label='ER')
plt.hist(df.query('species == 1')['ionizationE'],bins=899,range = (2,450),label='NR')
plt.yscale('log')
plt.xscale('log')
plt.xlabel(r'Ionization energy [$\rm keV_{ee}$]')
plt.ylabel('Counts')
plt.legend()

### Processing_images

Binned images are stored in "sparse" format as opposed to "dense" format. Sparse data includes (x,y,z,...) coordinate info for pixels > threshold value (we use 0 here). This means we ignore all 0's in our image.
This is computationally and storage efficient, as most of our image is empty (i.e. 0).

A 2048 x 1152 image of 16 bit pixels is 2048 x 1152 x 16 bits x 1 byte / 8 bits = 4.7 MB

On average, each image has around 7,000 non-zero pixels, so storing the image in sparse format gives:\
**2** & 7000 & 16 bits * 1byte / 8 bits = 28 kB **factor of 160 reduction over dense**

The bolded **2** comes from the fact that we're looking at x-y images, so we have 7000 16-bit integers for x and an additional 7000 16-bit integers for y

### Making images dense requires us binning them.
**It's actually computationally faster to bin sparse coordinates so sparsity helps us in many ways**

We'll be using [numpy's histogram2d function for this](https://numpy.org/doc/stable/reference/generated/numpy.histogram2d.html)

In [ ]:
df.query('truthE == 11.8')

In [ ]:
# converting sparse coordinates to dense coordinates. Use np.histogram2d

eventnum = 285 #let's look at event number 0

event = df.iloc[eventnum] #grab the event

#Declaring each argument for clarity. Native bin size is (2048,1152). xcam, and ycam
#are already binned to 0-2047 and 0-1151, respectively
#The function reference shows that argument 0 of the function is the 2D histogram (ndarray)

im = np.histogram2d(x=event['xcam'],y=event['ycam'],weights=event['qcam'],
                   bins=(2048,1152),range=((0,2048),(0,1152)))[0].T #transpose makes x and y as they should be

In [ ]:
np.shape(im) #image shape is what we want

In [ ]:
# Colormap reference https://matplotlib.org/stable/users/explain/colors/colormaps.html

#Our group uses 'jet' for historical reasons. We really should use a perceptually uniform sequential
#colormap like 'viridis' or 'plasma'
plt.imshow(im,cmap='jet')
plt.xlim(event['xcam'].min()-10,event['xcam'].max()+10)
plt.ylim(event['ycam'].min()-10,event['ycam'].max()+10)

In [ ]:
'''Lets make a function to efficiently bin images'''

#bin_factor is the factor we downsample the image by. For example
#bin_factor = 4 is 4x4 binning
def bin_image(i,bin_factor):
    tmp = df.iloc[i]
    im = np.histogram2d(x=tmp['xcam'],y=tmp['ycam'],weights=tmp['qcam'],
                   bins=(2048//bin_factor,1152//bin_factor),
                        range=((0,2048),(0,1152)))[0].T #transpose makes x and y as they should be
    return im

In [ ]:
bin_fact = 4
plt.imshow(bin_image(eventnum,bin_factor = bin_fact),cmap='jet')
plt.xlim((event['xcam'].min()-10)//bin_fact,(event['xcam'].max()+10)//bin_fact)
plt.ylim((event['ycam'].min()-10)//bin_fact,(event['ycam'].max()+10)//bin_fact)

### We've seen examples of plotting tracks, but for electron rejection, we want to come up with discriminates that can be used to reject ERs and keep NRs

As an example let's compute the length of tracks in about the most rudimentary way we can think of. We could simply compute the length of the diagonal of the smallest bounding box fully containing each track

In [ ]:
'''The smallest bounding box is the box whose perimeter is defined by the minimum x and y coordinates
Let"s compute the x and y extrema for each event'''

df['xmin'] = df['xcam'].apply(lambda x: x.min()) #minimum on camera readout
df['ymin'] = df['ycam'].apply(lambda x: x.min()) #minimum on camera readout
df['xmax'] = df['xcam'].apply(lambda x: x.max()) #maximum on camera readout
df['ymax'] = df['ycam'].apply(lambda x: x.max()) #maximum on camera readout

In [ ]:
'''Now lets compute this length and convert to mm'''

df['crude_length'] = np.sqrt((df['xmax']-df['xmin'])**2+(df['ymax']-df['ymin'])**2)*80/2048 #mm

In [ ]:
'''Just for fun lets plot this length on our track'''
event = df.iloc[eventnum]
plt.imshow(im,cmap='jet')
'''plot bounding box'''
plt.hlines(event['ymin'],event['xmin'],event['xmax'],color='red',lw=2) #bottom horiz line
plt.hlines(event['ymax'],event['xmin'],event['xmax'],color='red',lw=2) #top horiz line
plt.vlines(event['xmin'],event['ymin'],event['ymax'],color='red',lw=2) #left vert line
plt.vlines(event['xmax'],event['ymin'],event['ymax'],color='red',lw=2) #right vert line
'''plot length'''
plt.plot([event['xmin'],event['xmax']],[event['ymin'],event['ymax']],color='w')
plt.xlim(event['xcam'].min()-10,event['xcam'].max()+10)
plt.ylim(event['ycam'].min()-10,event['ycam'].max()+10)

### The above is a pretty poor approximation but its a start

Now lets see how well length does as a discriminant

In [ ]:
plt.hist(df.query('species == 0')['crude_length'],bins=101,range=(0,30),label='ER')
plt.hist(df.query('species == 1')['crude_length'],bins=101,range=(0,30),label='NR')
plt.xlabel('Length of bounding box diag [mm]')

### What if we add energy into the mix

In [ ]:
plt.plot(df.query('species == 0')['crude_length'],df.query('species == 0')['ionizationE'],'o',label = 'ER',markersize = 2)
plt.plot(df.query('species == 1')['crude_length'],df.query('species == 1')['ionizationE'],'o',label = 'NR',markersize = 2)
plt.xlabel('Length of bounding box diag [mm]')
plt.ylabel(r'Ionization energy [$\rm keV_{ee}$]')
plt.ylim(0,50)
plt.legend()

### Now we're getting somewhere!